In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [2]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_AM_ext.py

272
64
rank of S is: 

64
sizes of S are: 

64
64
rank of PA_t is: 

54
sizes of PA_t are: 

272
64
rank of Q is: 

54
sizes of Q are: 

272
272


/home/jzh/Dropbox/Research/Data-driven_estimation_inverse_optimization/Experiments/InverseVIsTraffic/Python_files/util.py:137: ComplexWarning: Casting complex values to real discards the imaginary part
  obj += (1.0 / 2) * K * lam[i] * Q[i, j] * lam[j]


GurobiError: Invalid argument to LinExpr multiplication

In [3]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_MD_ext.py

rank of S is: 

64
sizes of S are: 

64
64
rank of PA_t is: 

54
sizes of PA_t are: 

272
64
rank of Q is: 

54
sizes of Q are: 

272
272


GurobiError: Objective Q not PSD (diagonal adjustment of 9.4e+02 would be required)

In [4]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_PM_ext.py

rank of S is: 

64
sizes of S are: 

64
64
rank of Q is: 

54
sizes of Q are: 

272
272


GurobiError: Objective Q not PSD (diagonal adjustment of 8.8e+03 would be required)

In [5]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_NT_ext.py

rank of S is: 

64
sizes of S are: 

64
64
rank of Q is: 

54
sizes of Q are: 

272
272


GurobiError: Objective Q not PSD (diagonal adjustment of 9.8e+02 would be required)

In [6]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekend_ext.py

rank of S is: 

64
sizes of S are: 

64
64
rank of Q is: 

54
sizes of Q are: 

272
272


GurobiError: Objective Q not PSD (diagonal adjustment of 7.6e+02 would be required)

In [7]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_AM_ext.py

rank of S is: 

64
sizes of S are: 

64
64
rank of Q is: 

54
sizes of Q are: 

272
272


GurobiError: Objective Q not PSD (diagonal adjustment of 2.1e+02 would be required)

In [8]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_MD_ext.py

In [9]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_PM_ext.py

In [10]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_NT_ext.py

In [11]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekend_ext.py

In [12]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_AM_ext.py

In [13]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_MD_ext.py

In [14]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM_ext.py

In [15]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_NT_ext.py

In [16]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekend_ext.py

In [17]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_AM_ext.py

In [18]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_MD_ext.py

In [19]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_PM_ext.py

In [20]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_NT_ext.py

In [21]:
%run ../Python_files/util.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekend_ext.py